<a href="https://colab.research.google.com/github/taolati20011/INT3405/blob/main/preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
from google.colab import drive
drive.mount("/content/drive")
input = "/content/drive/MyDrive/BERT/train.csv"
output = "/content/drive/MyDrive/BERT/train_preprocess.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
import pandas as pd
pd.set_option('display.max_colwidth',2)
pd.set_option("display.max_rows", 200)
data = pd.read_csv(input)
data.head(5)

,id,text,class
0,dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng_rãi thoáng mát sạch_sẽ .\nNhân_viên phục_vụ nhiệt_tình , siêu dễ_thương .\nThực_đơn phong_phú đa_dạng , giá siêu rẻ .\nThức_ăn nêm nếm hợp_khẩu vị , ngon_miệng .\nDesert ngọt vừa , không quá ngấy , rất ngon .\nTrà sữa , uyên_ương thơm và đúng vị mình đã từng uống ở HongKong .\nNói_chung điểm 10 cho chất_lượng , thích quán lắm !",1
1,800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ổn . Tuy_nhiên không_gian tầng 1 hơi hẹp . Ngồi cả ra vỉa_hè . Đồ uống ko ngon lămz,0
2,6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi gần 15p mới mang được ly cà_phê ra . Ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa .,0
3,b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nên phục_vụ chưa tốt . Đồ uống bình_thường ( đặc_biệt khi so với giá_cả ) và ko có gì đặc_sắc cả .",0
4,f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiếp_đãi , tư_vấn và nói_chuyện rất gần_gũi , rất nhiệt_tình .\nBánh thì rất tuyệt , nhân béo , rau_câu không dùng phẩm . Sau_này có dịp sinh_nhật , mình sẽ order mẫu lớn .\nChúc các bạn làm_ăn phát_đạt !",1


In [123]:
from math import log
from unicodedata import normalize
import re
import string

words_path = "/content/drive/MyDrive/words.txt"
words = open(words_path).read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

# Split text without space into list of word
# Based on: https://stackoverflow.com/questions/8870261/how-to-split-text-without-spaces-into-list-of-words/11642687#11642687
def infer_spaces(s):
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

def rm_unicode(str):
    # Lower text
    str = str.lower()

    str = re.sub("[,.;:@#?!&$_/-]", " ", str)

    # Find invalid unicode 
    if re.search(b"( )(\xcc[\x80-\xa9])( )", str.encode('utf-8')):
        # Change str to normal form
        str = re.sub(b"( )(\xcc[\x80-\xa9])( )", br'\2', str.encode('utf-8')).decode('utf-8')

    word_list = str.split()
    new_list = []

    for i, w in enumerate(word_list):
        # Find stuck words & divide them
        if len(re.findall(b"\xcc", w.encode('utf-8'))) > 0:
            w = normalize("NFKC", w)
            w = infer_spaces(w)
            new_list.append(w)
        else:
            new_list.append(w)


    text = " ".join(new_list)
    return text

In [124]:
def preprocess_text(str):
    str = rm_unicode(str)
    # Remove html tag
    CLEANR = re.compile('<.*?>') 
    str = re.sub(CLEANR, '', str)

    # Remove emoji
    # Based on: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u2022"  # bullet point
                      "]+", re.UNICODE)
    str = emoji_pattern.sub(r'', str) # no emoji
    
    # Remove "_","-" & "/", ...
    # mặt_đường -> mặt đường & 9-10h -> 9 10h & 50k/tô -> 50k tô
    str = re.sub("[,.;:@#?!&$_/-]", " ", str)

    # Remove punctuation
    # Based on: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    exclude = set(string.punctuation)
    str = ''.join(ch for ch in str if ch not in exclude)

    # Remove double space
    str = str.replace("  ", " ")

    # Remove space in the first and last position
    while (str[0]==" "):
        str = str[1:]
    while (str[len(str)-1] == " "):
        str = str[:len(str)-1]

    # Remove same characters in last position each word
    word_list = str.split()
    new_list = []
    for i in word_list:
        if (len(i) < 2):
            new_list.append(i)
            continue
        while (i[len(i)-1] == i[len(i)-2]):
            i = i[:len(i)-1]
            if (len(i) < 2):
                break
        new_list.append(i)

    str = " ".join(new_list)
    return str

s = "Hôm mi ̀ nh đê ́ n qua ́ n kha ́ vă ́ ng kha ́ ch .\nMi ̀ nh đi hai ng co ́ go ̣ i hai bu ́ n ô ́ c va ̀ 3ba ́ nh tôm hê ́ t 150k.lu ́ c thanh toa ́ n ma ̀ mi ̀ nh giâ ̣ t mi ̀ nh luôn . ho ̉ i thi ̀ ba ́ c ba ́ n ha ̀ ng ba ̉ o 50k/ba ́ t bu ́ n ô ́ c.ba ba ́ nh tôm la ̀ 50k @ @ .\nBa ̉ n thân mi ̀ nh la ̀ sinh_viên nên thâ ́ y kha ́ đă ́ t trong khi châ ́ t lươ ̣ ng cu ̃ ng k đươ ̣ c tô ́ t ."

In [125]:
data['preprocess_text'] = data['text'].apply(lambda x: preprocess_text(x))
data.head(10)

,id,text,class,preprocess_text
0,dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng_rãi thoáng mát sạch_sẽ .\nNhân_viên phục_vụ nhiệt_tình , siêu dễ_thương .\nThực_đơn phong_phú đa_dạng , giá siêu rẻ .\nThức_ăn nêm nếm hợp_khẩu vị , ngon_miệng .\nDesert ngọt vừa , không quá ngấy , rất ngon .\nTrà sữa , uyên_ương thơm và đúng vị mình đã từng uống ở HongKong .\nNói_chung điểm 10 cho chất_lượng , thích quán lắm !",1,đến quán 2 lần thôi rất là thích quán tuy nằm trong hẻm nhưng rất dễ tìm quán rộng rãi thoáng mát sạch sẽ nhân viên phục vụ nhiệt tình siêu dễ thương thực đơn phong phú đa dạng giá siêu rẻ thức ăn nêm nếm hợp khẩu vị ngon miệng desert ngọt vừa không quá ngấy rất ngon trà sữa uyên ương thơm và đúng vị mình đã từng uống ở hongkong nói chung điểm 10 cho chất lượng thích quán lắm
1,800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ổn . Tuy_nhiên không_gian tầng 1 hơi hẹp . Ngồi cả ra vỉa_hè . Đồ uống ko ngon lămz,0,đến quán vào tối chủ nhật có band hát khá ổn tuy nhiên không gian tầng 1 hơi hẹp ngồi cả ra vỉa hè đồ uống ko ngon lămz
2,6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi gần 15p mới mang được ly cà_phê ra . Ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa .,0,phục vụ lâu quá mặc dù khách rất vắng đợi gần 15p mới mang được ly cà phê ra ông này chỉ khéo quảng bá chứ với mình chắc không quay lại nữa
3,b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nên phục_vụ chưa tốt . Đồ uống bình_thường ( đặc_biệt khi so với giá_cả ) và ko có gì đặc_sắc cả .",0,ko gian bé tí quán chật chội đông người nên phục vụ chưa tốt đồ uống bình thường đặc biệt khi so với giá cả và ko có gì đặc sắc cả
4,f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiếp_đãi , tư_vấn và nói_chuyện rất gần_gũi , rất nhiệt_tình .\nBánh thì rất tuyệt , nhân béo , rau_câu không dùng phẩm . Sau_này có dịp sinh_nhật , mình sẽ order mẫu lớn .\nChúc các bạn làm_ăn phát_đạt !",1,khi mình order đặt bánh thì nhận được sự tiếp đãi tư vấn và nói chuyện rất gần gũi rất nhiệt tình bánh thì rất tuyệt nhân béo rau câu không dùng phẩm sau này có dịp sinh nhật mình sẽ order mẫu lớn chúc các bạn làm ăn phát đạt
5,8fea8db2,"An ở đây lần thứ n mà giờ mới review . Quán lúc_nào cũng đông . Nhân_viên chạy ra chạy vô tội_nghiệp luôn . Mọi thứ viên chiên ở đây có đủ . Mỗi lần thèm an vặt là mình lại ra đây . Giá khá rẻ . 1 dĩa an phủ_phê luôn 4 đứa chia ra mỗi đứa 50k . Lúc trước an thì quán chỉ bán ở nhà chính thôi , riết rồi lấn ra đuờng rồi qua bên quán cơm tấm 3 lùn bán luôn . Phát_triển ghê_gớm",0,an ở đây lần thứ n mà giờ mới review quán lúc nào cũng đông nhân viên chạy ra chạy vô tội nghiệp luôn mọi thứ viên chiên ở đây có đủ mỗi lần thèm an vặt là mình lại ra đây giá khá rẻ 1 dĩa an phủ phê luôn 4 đứa chia ra mỗi đứa 50k lúc trước an thì quán chỉ bán ở nhà chính thôi riết rồi lấn ra đuờng rồi qua bên quán cơm tấm 3 lùn bán luôn phát triển ghê gớm
6,2db8ae0a,"Quán rất đẹp . Nhạc rất hay . Rất art ! ! Bước vô là thích liền luôn : 3 Đồ uống thì buổi_sáng hôm đó mình đi thì_có cà_phê à , nên uống bạc xỉu đỡ : ' ( phục_vụ dễ_thương . Nói_chung quán yên_tĩnh và đẹp lắm , mình thích nhưng nơi yên_tĩnh như_vậy . Sẽ nhất_định quay lại Người_Tình . À , trên bàn dài có cuốn note bự thiệt bự , dành cho mọi người thích ghi những kỉ_niệm đến quán hay của riêng mình vào ... Có nhiều người vẽ đẹp lắm , hi . Khen mãi quán thôi , chả có gì chê luôn . = ) )",1,quán rất đẹp nhạc rất hay rất art bước vô là thích liền luôn 3 đồ uống thì buổi sáng hôm đó mình đi thì có cà phê à nên uống bạc xỉu đỡ phục vụ dễ thương nói chung quán yên tĩnh và đẹp lắm mình thích nhưng nơi yên tĩnh như vậy sẽ nhất định quay lại người tình à trên bàn dài có cuốn note bự thiệt bự dành cho mọi người thích ghi những kỉ niệm đến quán hay của riêng mình vào có nhiều người vẽ đẹp lắm hi khen mãi quán thôi chả có gì chê luôn
7,5a840374,"Gemini thì nhạc bao hay òy : 3 Dj đánh là lên nóc luôn ... Lên là lên là lên đụng nóc luônnn ... Cập_nhậ

In [126]:
data.to_csv("train_preprocess.csv", columns=['id', 'preprocess_text', 'class'])
!cp "train_preprocess.csv" $output


In [127]:
old_data = "/content/drive/MyDrive/shopee-sentiment/train_preprocess_unsegment.csv"
read_old_data = pd.read_csv(old_data)
data['old_preprocess_text'] = read_old_data['preprocess_text']

import numpy as np
data['compare'] = np.where((data['preprocess_text'] == data['old_preprocess_text']), 1, 0)
data['compare'].value_counts()

0    14289
1    12711
Name: compare, dtype: int64

In [129]:
data.sample(10)

,id,text,class,preprocess_text,old_preprocess_text,compare
26684,11cffcd6,"Đồ_ăn nguội , không ngon cho lắm , không_gian thì chật nhưng chỉ_được_cái là nhìn rất đúng chất với thị . Đồ_ăn hết thì không thay , giữ xe thì kiểu ngồi chơi , chứ chả quan_tâm khách .\nNói_chung không nên đến . Hôm mình đi , tận 12 đứa , mình chịu trách_nhiệm mua voucher mà đến_nơi đứa nào cũng chê nên ngại lắm",0,đồ ăn nguội không ngon cho lắm không gian thì chật nhưng chỉ được cái là nhìn rất đúng chất với thị đồ ăn hết thì không thay giữ xe thì kiểu ngồi chơi chứ chả quan tâm khách nói chung không nên đến hôm mình đi tận 12 đứa mình chịu trách nhiệm mua voucher mà đến nơi đứa nào cũng chê nên ngại lắm,đồ ăn nguội không ngon cho lắm không gian thì chật nhưng chỉ được cái là nhìn rất đúng chất với thị đồ ăn hết thì không thay giữ xe thì kiểu ngồi chơi chứ chả quan tâm khách nói chung không nên đến hôm mình đi tận 12 đứa mình chịu trách nhiệm mua voucher mà đến nơi đứa nào cũng chê nên ngại lắm,1
17576,c5fdf5d2,"Ngon lắm . Mấy chị_em mình ai cũng thích , toàn đặt ship . Giá_cả vừa_phải , bánh ăn_không ớn . Thích nhất nama chocolate",1,ngon lắm mấy chị em mình ai cũng thích toàn đặt ship giá cả vừa phải bánh ăn không ớn thích nhất nama chocolate,ngon lắm mấy chị em mình ai cũng thích toàn đặt ship giá cả vừa phải bánh ăn không ớn thích nhất nama chocolate,1
11848,6b0f7307,Mình cảm_thấy thức_ăn nhanh đều na_ná như nhau nhưng nếu ăn hambuger mình sẽ chọn burger king - rẻ hoqn và ngon hơn . Rất ấn_tượng apple pie của MD - không_bao_giờ thử dù chỉ 1 lần,0,mình cảm thấy thức ăn nhanh đều na ná như nhau nhưng nếu ăn hambuger mình sẽ chọn burger king rẻ hoqn và ngon hơn rất ấn tượng apple pie của md không bao giờ thử dù chỉ 1 lần,mình cảm thấy thức ăn nhanh đều na ná như nhau nhưng nếu ăn hambuger mình sẽ chọn burger king rẻ hoqn và ngon hơn rất ấn tượng aple pie của md không bao giờ thử dù chỉ 1 lần,0
14873,83320921,"Pizza ngon nhưng giá lại rẻ , mỳ ý ở đây mình cũng duyệt luôn , vs mình ngon , vừa ăn , thêm món salad gà nữa , mình thích nhất món này ở đây , cứ đến là gọi , nước uống thì tạm ổn . Mấy anh nhân_viên ở đây cũng rất nhiệt_tình , vui_tính . Từ ngày biết quán này , mỗi lần muốn ăn pizza toàn đến đây , chậc chậc ☺ ️ ☺ ️",1,pizza ngon nhưng giá lại rẻ mỳ ý ở đây mình cũng duyệt luôn vs mình ngon vừa ăn thêm món salad gà nữa mình thích nhất món này ở đây cứ đến là gọi nước uống thì tạm ổn mấy anh nhân viên ở đây cũng rất nhiệt tình vui tính từ ngày biết quán này mỗi lần muốn ăn pizza toàn đến đây chậc chậc,piza ngon nhưng giá lại rẻ mỳ ý ở đây mình cũng duyệt luôn vs mình ngon vừa ăn thêm món salad gà nữa mình thích nhất món này ở đây cứ đến là gọi nước uống thì tạm ổn mấy anh nhân viên ở đây cũng rất nhiệt tình vui tính từ ngày biết quán này mỗi lần muốn ăn piza toàn đến đây chậc chậc,0
23185,f106af16,"Nghe bạn_bè chỉ và tin_đồn trên facebook này_nọ nên đã tìm tới qua nhưng hoàn_toàn thất_vọng , tok thì bột bở bở được mấy cục dai dai ăn được ( còn dở hơn bánh gạo mình mua trên lotte về tự nấu nữa ) gà thì phần nạt có chút_xíu ngoài_ra toàn phủ lớp bột khá to , sốt cũng bt chẳng cứu_vãn được gì , ăn thử thì nên nhưng quay lại ủng_hộ thì không",0,nghe bạn bè chỉ và tin đồn trên facebook này nọ nên đã tìm tới qua nhưng hoàn toàn thất vọng tok thì bột bở bở được mấy cục dai dai ăn được còn dở hơn bánh gạo mình mua trên lotte về tự nấu nữa gà thì phần nạt có chút xíu ngoài ra toàn phủ lớp bột khá to sốt cũng bt chẳng cứu vãn được gì ăn thử thì nên nhưng quay lại ủng hộ thì không,nghe bạn bè chỉ và tin đồn trên facebok này nọ nên đã tìm tới qua nhưng hoàn toàn thất vọng tok thì bột bở bở được mấy cục dai dai ăn được còn dở hơn bánh gạo mình mua trên lote về tự nấu nữa gà thì phần nạt có chút xíu ngoài ra toàn phủ lớp bột khá to sốt cũng bt chẳng cứu vãn được gì ăn thử thì nên nhưng quay lại ủng hộ thì không,0
11745,1b689980,"Ai khen ngon thì mình ko bik chứ theo mình thì uống tệ thật . đi cùng bạn đến , kêu 1 ly trà sữa nho và chanh dây đào . Trà sữa nhạt